# Testing Snowplow Algorithm

Consideration: if no unserviced edges except for u-turning, consider probabilistically choosing to traverse a serviced edge to escape a local area.

In [1]:
import os
%load_ext autoreload
%autoreload 2

In [2]:
import Snowplow_Routing_Middleton.dual_graphs as dual_graphs
import Snowplow_Routing_Middleton.construction as construction
import Snowplow_Routing_Middleton.plotting as plotting
import Snowplow_Routing_Middleton.local_search as local_search
import Snowplow_Routing_Middleton.initialize as initialize
import Snowplow_Routing_Middleton.params as params
import Snowplow_Routing_Middleton.sectioning as sectioning
import Snowplow_Routing_Middleton.costs as costs


from Snowplow_Routing_Middleton.shortest_paths import ShortestPaths
from Snowplow_Routing_Middleton.crossover import apply_crossover
from Snowplow_Routing_Middleton.genetic import run_genetic
from Snowplow_Routing_Middleton.solution import Solution
from Snowplow_Routing_Middleton.costs import routes_cost, calculate_time_deadheading
from Snowplow_Routing_Middleton.routes_representations import create_full_routes_with_returns
from Snowplow_Routing_Middleton.turns import turn_direction_count

import pandas as pd
import osmnx as ox
import folium
import folium.plugins
import networkx as nx
import matplotlib.pyplot as plt


Autoreload documentation: https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html

For reloading user modules

In [3]:
toyEdgeFile = "C:\\Users\\Sneez\\Desktop\\Snowplowing\\Snowplow-Routing-Middleton\\Snowplow-Routing-Middleton\\graph_data\\edges.csv"
toyNodeFile = "C:\\Users\\Sneez\\Desktop\\Snowplowing\\Snowplow-Routing-Middleton\\Snowplow-Routing-Middleton\\graph_data\\nodes.csv"

In [4]:
curr_path = os.path.abspath("")
parent_path = os.path.split(curr_path)[0]

In [20]:
instance = "sectionedfullstreets"

polygon_path = os.path.join(parent_path, "graph_data\\individual_sections\\route4_red.gpkg")

match instance:
    case "smallstreets":
        G = initialize.create_small_streets()
    case "fullstreets":
        G = initialize.create_full_streets()
    case "smalltoy":
        G = initialize.create_small_toy(toyEdgeFile, toyNodeFile)
    case "sectionedfullstreets":
        G = sectioning.section_component(polygon_path, True)
    case _:
        print("Invalid instance name")
     

In [21]:
ox.graph_to_gdfs(G, nodes=False, edges=True).explore()

Update depot

In [22]:
if instance == "fullstreets" or instance == "sectionedfullstreets":
    params.DEPOT = params.find_depot(G)[0]
else:
    params.DEPOT = 0
params.DEPOT

277

In [23]:
approach = "genetic"

if approach == "genetic":
    G = initialize.add_multi_edges(G)
    if instance == "smallstreets" or instance == "fullstreets" or instance == "sectionedfullstreets":
        G_DUAL = dual_graphs.create_dual_streets(G, params.DEPOT)
    else:
        G_DUAL = dual_graphs.create_dual_toy(G)

elif approach == "annealing":
    if instance == "smallstreets" or instance == "fullstreets" or instance == "sectionedfullstreets":
        G_DUAL = dual_graphs.create_dual_streets(G, False, True)
    else:
        G_DUAL = dual_graphs.create_dual_toy(G, False, True)



Adding depot source 277


In [24]:
print(G)
print(G_DUAL)

MultiDiGraph with 285 nodes and 1163 edges
MultiDiGraph with 1164 nodes and 5965 edges


In [25]:
shortest_paths_folderpath  = os.path.join(parent_path, "graph_data\\shortest_paths\\route4_red")


In [26]:
sp = ShortestPaths(G_DUAL, False, True, shortest_paths_folderpath)